In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_climat_par_saison(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    # Trouver le tableau ciblé (le plus spécifique possible)
    table = soup.select_one("table.odTable.odTableAuto")
    data = []

    if table:
        # En-têtes des colonnes
        headers = [th.get_text(strip=True) for th in table.find("tr").find_all("th")]
        
        for row in table.find_all("tr")[1:]:
            cols = row.find_all(["td", "th"])
            data.append([col.get_text(strip=True) for col in cols])
        
        df = pd.DataFrame(data, columns=headers)
        return df
    else:
        print("❌ Tableau introuvable")
        return None

                                   Hiver Printemps       Eté   Automne
0                       Soleil                                        
1      Heures d'ensoleillement     259 h     521 h     744 h     233 h
2            Moyenne nationale     290 h     542 h     656 h     264 h
3   Equivalent jours de soleil      11 j      22 j      31 j      10 j
4            Moyenne nationale      12 j      23 j      27 j      11 j
5                        Pluie                                        
6             Hauteur de pluie    303 mm    386 mm    257 mm    314 mm
7            Moyenne nationale    258 mm    238 mm    208 mm    234 mm
8                         Vent                                        
9     Vitesse de vent maximale   83 km/h   76 km/h   83 km/h  101 km/h
10           Moyenne nationale  169 km/h  151 km/h  130 km/h  155 km/h


In [5]:
# Lire par chunks
chunksize = 50
reader = pd.read_csv("communes-france-2025.csv", dtype={"code_insee": str}, low_memory=False, chunksize=chunksize)

data_concat = []

for chunk_index, df_chunk in enumerate(reader):
    # Supprimer première colonne si elle est inutile
    if df_chunk.columns[0].startswith("Unnamed"):
        df_chunk = df_chunk.drop(df_chunk.columns[0], axis=1)

    # Filtrer les villes > 20k habitants
    filtered_source = df_chunk[df_chunk["population"] >= 20000]

    for i in range(filtered_source.shape[0]):
        code_insee = filtered_source["code_insee"].iloc[i]
        ville = filtered_source["nom_sans_accent"].iloc[i]

        url = f"https://www.linternaute.com/voyage/climat/{ville}/ville-{code_insee}"
        recherche = get_climat_par_saison(url)
        if recherche is not None:
            recherche["code_insee"] = code_insee
            recherche["Ville"] = ville
            data_concat.append(recherche)

            print("✅ Ligne ajoutée au DataFrame :")
            print(f"{ville} : {code_insee}")
            print("------------------------------------------------------------")

print("Recherches terminées")
# Concat final
df_final = pd.concat(data_concat, ignore_index=True)

df_final.to_csv("climat_saison.csv", index=False, encoding="utf-8-sig")

✅ Ligne ajoutée au DataFrame :
bourg-en-bresse : 01053
------------------------------------------------------------
✅ Ligne ajoutée au DataFrame :
oyonnax : 01283
------------------------------------------------------------
✅ Ligne ajoutée au DataFrame :
laon : 02408
------------------------------------------------------------
✅ Ligne ajoutée au DataFrame :
saint-quentin : 02691
------------------------------------------------------------
✅ Ligne ajoutée au DataFrame :
soissons : 02722
------------------------------------------------------------
✅ Ligne ajoutée au DataFrame :
montlucon : 03185
------------------------------------------------------------
✅ Ligne ajoutée au DataFrame :
vichy : 03310
------------------------------------------------------------
✅ Ligne ajoutée au DataFrame :
manosque : 04112
------------------------------------------------------------
✅ Ligne ajoutée au DataFrame :
gap : 05061
------------------------------------------------------------
✅ Ligne ajoutée au 